# IQVIA NLP - FDA Drug Label Biomarkers

## API Description
Understanding biomarkers for drug efficacy or disease progression is critical in clinical development. Relevant data is often in unstructured text, such as clinical trial records, and drug labels.   For Data Scientists, the FDA Drug Label Biomarkers API solves this problem by offering an API that extracts gene/protein biomarkers from specific regions of FDA Drug labels (Indications and Usages section). Biomarker-disease relationships are provided (e.g.positive, negative); diseases are mapped to MedDRA, and mechanism of action data are also extracted.

## Accessing the API
In order to consume this API, you will first need to Request access to the FDA Drug Label Biomarkers API 
via this link:
https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytJqMAAU/api-marketplaceiqvianlpfdadruglabelbiomarkerspreview .

Please refer to "API Documentation" to learn more about accessing and using the API.

## Notebook Description
This notebook is designed to biomarkers and their related diseases from from openFDA drug product labeling records. Users could specify records of interest by supplying FDA Drug Label document identifiers in the 'FDAID' parameter when posting the request.

### Authorization
The instructions for getting your credentials and the API endpoint URL can be found under the section "Get Started" and "How to use the API" following this link: https://api-marketplace.work.iqvia.com/s/communityapi/a085w00000ytJqMAAU/api-marketplaceiqvianlpfdadruglabelbiomarkerspreview .

In [1]:
import getpass
import requests

# In this demo scenario, URL for US based customers
# api_marketplace_url = 'https://vt.us-rds.solutions.iqvia.com/fda/api/v1/biomarkers'
# In this demo scenario, URL for EU based customers
api_marketplace_url = 'https://vt.eu-apim.solutions.iqvia.com/eu/fda/api/v1/biomarkers'

mkp_user = input("Marketplace clientId: ")
mkp_password = getpass.getpass("Marketplace clientSecret: ")
mkp_headers = {'clientId': mkp_user, 'clientSecret': mkp_password}

# Check credentials by making a dummy request
print("Checking your credentials, please wait...")
response = requests.get(api_marketplace_url, headers=mkp_headers)

if response.status_code == 200:
    print("Congratulations! Your credentials are accepted!")
else:
    raise Exception(f"Error: {response}")

Marketplace clientId: 77922ba7e417474a959a705fee4f272f
Marketplace clientSecret: ········
Checking your credentials, please wait...
Congratulations! Your credentials are accepted!


### Example: Make a request for a list of known FDA Drug Label documents
FDA Drug Label Biomarker NLP API expects openFDA application number (FDAID) as input parameter. This example shows how to make a request to the API with a list of FDAIDs.


In [3]:
import requests

# Define input FDAIDs
input_fdaids = ['NDA204114', 'ANDA076183']

# Define placeholder for output
results_json = []

# Make a request
print("Posting request to extract biomarkers and related diseases from specified CT.gov documents...")

for fdaid in input_fdaids:
    print(f"Processing {fdaid}...")
    response = requests.get(api_marketplace_url, headers=mkp_headers, params={'FDAID': fdaid})
    # Check the response
    if response.status_code == 200:
        print("Success!")
        body_json = response.json()
        results_json.append(body_json)
    else:
        raise Exception(f"Error: {response}")

print(results_json)

Posting request to extract biomarkers and related diseases from specified CT.gov documents...
Processing NDA204114...
Success!
Processing ANDA076183...
Success!
[[{'Biomarker': 'ABCB1', 'Relation': 'inhibitor', '[NID] MedDRA': '', '[PT] MedDRA': '', 'Section': 'Clinical Pharmacology', 'FDAID': 'NDA204114', 'Label Revision Date': '20220622', 'Source': 'https://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=0002ad27-779d-42ab-83b5-bc65453412a1', 'Hit': '... , trametinib is not an inhibitor of P-gp, BCRP, OATP1B1, ...'}, {'Biomarker': 'ABCB1', 'Relation': 'inhibitor', '[NID] MedDRA': '', '[PT] MedDRA': '', 'Section': 'Other Section', 'FDAID': 'NDA204114', 'Label Revision Date': '20220622', 'Source': 'https://dailymed.nlm.nih.gov/dailymed/lookup.cfm?setid=0002ad27-779d-42ab-83b5-bc65453412a1', 'Hit': '... , trametinib is not an inhibitor of P-gp, BCRP, OATP1B1, ...'}, {'Biomarker': 'ABCG2', 'Relation': 'inhibitor', '[NID] MedDRA': '', '[PT] MedDRA': '', 'Section': 'Clinical Pharmacology', 

Now that we have got the JSON responses from the FDA Drug Label Biomarker API, we could convert the useful information associated with the keys into a pandas dataframe.

In [4]:
import pandas as pd

# initiate an empty dataframe
df = pd.DataFrame()
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.width", 1000)

# Retrieve main results from the JSON response, please note this cell would fail if the request failed in the last step
for results in results_json:
    for result_dict in results:
        df_dict = {}
        for key, value in result_dict.items():
            df_dict[key] = value
        df = pd.concat([df, pd.DataFrame.from_records([{**df_dict}])], ignore_index=True)

# Check the dataframe
df

,Biomarker,Relation,[NID] MedDRA,[PT] MedDRA,Section,FDAID,Label Revision Date,Source,Hit
0,ABCB1,inhibitor,,,Clinical Pharmacology,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... , trametinib is not an inhibitor of P-gp, ..."
1,ABCB1,inhibitor,,,Other Section,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... , trametinib is not an inhibitor of P-gp, ..."
2,ABCG2,inhibitor,,,Clinical Pharmacology,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... , trametinib is not an inhibitor of P-gp, ..."
3,ABCG2,inhibitor,,,Other Section,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... , trametinib is not an inhibitor of P-gp, ..."
4,ALK,no mutation,,,Clinical Study,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... or MEK inhibitor, and absence of EGFR muta..."
5,ALK,no mutation,,,Other Section,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,"... or MEK inhibitor, and absence of EGFR muta..."
6,ALK,rearrangement,,,Clinical Study,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,... absence of EGFR mutation or ALK rearrangem...
7,ALK,rearrangement,,,Other Section,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,... absence of EGFR mutation or ALK rearrangem...
8,BRAF,activation,,,Clinical Pharmacology,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,... V600E mutations result in constitutive act...
9,BRAF,activation,,,Other Section,NDA204114,20220622,https://dailymed.nlm.nih.gov/dailymed/lookup.c...,... V600E mutations result in constitutive act...


### Optional: Make a request to openFDA drug product labeling API to get FDAIDs associated with a specific criteria

Search URL for the openFDA drug product labeling API that allows user to search for the FDAId's of 10 documents. Search can be customized by modifying the search parameters. In this example, the search will return 10 FDAIds of FDA Drug Labels about prescription drugs with indications and usage for cancer.


In [5]:
import json

# Search URL for the openFDA drug product labeling API
# Search can be customized by modifying the search parameters in the URL

url = "https://api.fda.gov/drug/label.json?search=openfda.product_type:prescription+AND+indications_and_usage:'cancer'&limit=10"

# load in JSON reponse from the openFDA API
response = requests.get(url)
body = json.loads(response.text)

# initialize list for FDAID
fdaids = []

# find the openfda application number in response (if exists) and add to FDAID list
for i in body['results']:
    try:
        #extract the document id (ANDA, ANADA, BA, BLA, BN, BP, DMF,K, MIF, NDA, NADA, P or VM)
        x = i["openfda"]
        fdaids.append(x['application_number'][0])
    except KeyError:
        print(f"No openFDA application number found for response results id: {i['id']}")
        continue

print(fdaids)

No openFDA application number found for response results id: 8cdcf814-f5ee-43b6-bf09-68eab5e5befe
['NDA204114', 'ANDA075798', 'ANDA040584', 'NDA050629', 'ANDA204345', 'NDA021116', 'NDA022234', 'ANDA080352', 'ANDA211495']


That's it! Hope you find this tutorial useful! Bye!